#MVP Engenharia de Dados - 2025
Thomas A. S. Abrantes


### Setup para rodar o notebook com os dados
- Imports das bibliotecas para consumo dos dados.

In [0]:
#bibioteca para uso do comando com Pandas
import pandas as pd

#biblioteca para carga de dados e request dos dados
import io
import requests

#Camada Silver<center>

**OBSERVAÇÃO**:  
- Nesta versão do databricks ele não permite a criação de chave primária e estrangeira. 
- Desta forma na camada SILVER os campos CHAVE da tabela são NOT NULL evitando a carga de valor nulo e CHAVE duplicada.

Para a camada SILVER será criada o banco de dados (schema) IMDB_DB_SILVER com tratamento dos dados das tabelas criadas em IMDB_DB_BRONZE.
Na migração dos dados, as tabelas permanecem com mesmo nome no schema IMDB_DB_SILVER mas com os seguintes tratamentos dos dados:

- Criar tabelas com o tipo das colunas de acordo com a documentação no dicionário de dados.
- Remover a legenda "\N" em algumas colunas que indica que o valor "não é aplicável".
- Validar chaves duplicadas se houver.
- Validar a quantidade de registros importados para o IMDB_DB_SILVER silver em comparação com o IMDB_DB_BRONZE.
- Tratamentos específicos em algumas tabelas se for necessário e se houver será informado abaixo.



In [0]:
%sql
-- Removendo Schema para recriar
DROP SCHEMA IF EXISTS IMDB_DB_SILVER CASCADE;

-- CRIANDO O BANCO DE DADOS (schema)
CREATE SCHEMA IMDB_DB_SILVER;

###Criando e preenchendo a "**tb_title.akas.tsv**".

- Colunas da tabela com tipo de acordo com dicionário de dados.
- Tratando os campos que do tipo string (texto) e tenha valor "\N" (não aplicável) com valor [vazio].
- Seguindo a documentação composta no dicionário, convertendo as colunas 'ordering' e 'isOriginalTitle' para o tipo numérico e inteiro e preenchendo com 0 (Zero) se o valor for nulo.
- Formatação na coluna "region".
- Formatação nas colunas "language" e "attributes".
- Tratamento de valores nulos com dataFrame criado na camada Bronze.
- Prenchendo a tabela com valores tratados com origem da tabela IMDB_DB_BRONZE.tb_title_akas na camada SILVER

In [0]:
#Removendo vínculo do arquivo da tabela
dbutils.fs.rm("dbfs:/user/hive/warehouse/imdb_db_silver.db/tb_title_akas/", True)

Out[58]: False

In [0]:
%sql
-- Criando a tabela "tb_title_akas"
CREATE OR REPLACE TABLE IMDB_DB_SILVER.tb_title_akas (
    titleId CHAR(10) NOT NULL,
    ordering INT NOT NULL,
    title VARCHAR(1000) NOT NULL,
    region CHAR(5),
    language CHAR(3),
    types VARCHAR(50),
    attributes VARCHAR(150),
    isOriginalTitle INT NOT NULL 
);

INSERT INTO IMDB_DB_SILVER.tb_title_akas (
    titleId,
    ordering,
    title,
    region,
    language,
    types,
    attributes,
    isOriginalTitle
)
SELECT 
    titleId,
    ordering,
    title,
    CASE WHEN TRIM(region) = '\\N' THEN NULL ELSE region END region,
    CASE WHEN TRIM(language) = '\\N' THEN NULL ELSE language END language,
    CASE WHEN TRIM(types) = '\\N' THEN NULL ELSE types END types,
    CASE WHEN TRIM(attributes) = '\\N' THEN NULL ELSE attributes END attributes,
    CASE WHEN isOriginalTitle IS NULL THEN 0 ELSE isOriginalTitle END isOriginalTitle
FROM IMDB_DB_BRONZE.tb_title_akas;

num_affected_rows,num_inserted_rows
51734703,51734703


In [0]:
%sql
-- LISTANDO OS 30 REGISTROS PARA NÃO FICAR MUITO GRANDE NO GITHUB E COMPROVAR A IMPORTAÇÃO
SELECT * FROM IMDB_DB_SILVER.tb_title_akas ORDER BY isOriginalTitle DESC LIMIT 30;

titleId,ordering,title,region,language,types,attributes,isOriginalTitle
tt0000005,1,Blacksmith Scene,null,null,original,null,1
tt7161844,1,Old Friends,null,null,original,null,1
tt1014087,1,Tesouro da Juventude,null,null,original,null,1
tt7161906,1,Chamber Check: Evolution of a Fight Scene,null,null,original,null,1
tt26341628,1,Episode #1.125,null,null,original,null,1
tt7161846,1,Retro Wick: Exploring the Unexpected Success of 'John Wick',null,null,original,null,1
tt1014092,1,Episode #4.7,null,null,original,null,1
tt7161828,1,Incertezas Críticas,null,null,original,null,1
tt14842066,1,Zero Legacy,null,null,original,null,1
tt7161848,1,Of Mice and Mo,null,null,original,null,1


Validar os dados importados para a camada SILVER

In [0]:
%sql
-- Verificar duplicidade.
-------------------------
SELECT CASE WHEN COUNT(0) > 1 THEN 'Há dados duplicados na tabela' ELSE 'Nenhum dado duplicado foi encontrado com a consulta' END AS Resultado 
FROM (
  SELECT titleId, COUNT(0) TOTAL FROM IMDB_DB_SILVER.tb_title_akas GROUP BY titleId HAVING COUNT(0) > 1
);

Resultado
Há dados duplicados na tabela


In [0]:
%sql
-- Conferência dasa quantidades de registros
--------------------------------------------
SELECT 
  (SELECT COUNT(0) FROM imdb_db_bronze.tb_title_akas) TOTAL_CAMADA_BRONZE,
  COUNT(0) TOTAL_CAMADA_SILVER, 
  CASE WHEN COUNT(0) = (SELECT COUNT(0) FROM imdb_db_bronze.tb_title_akas) THEN  'OK' ELSE 'ERRO - QUANTIDADE DE REGISTROS NA CAMADA SILVER DIFERENTE' END AS VERIFICACAO 
FROM IMDB_DB_SILVER.tb_title_akas;

TOTAL_CAMADA_BRONZE,TOTAL_CAMADA_SILVER,VERIFICACAO
51734703,51734703,OK


###Criando e preenchendo a "**tb_name_basics**".

- Colunas da tabela com tipo de acordo com dicionário de dados.
- Somente registros com chave não nula 
- Seguindo a documentação composta no dicionário, convertendo as colunas 'birthYear ' e 'deathYear ' para o tipo numérico e inteiro e preenchendo com 0 (Zero) se o valor for nulo.
- Prenchendo a tabela com valores tratados com origem da tabela IMDB_DB_BRONZE.tb_name_basics na camada SILVER
- birthYear e deathYear com valores do tipo inteiro e se nulo ou "\N" com valor zero

In [0]:
#Removendo vínculo do arquivo da tabela
dbutils.fs.rm("dbfs:/user/hive/warehouse/imdb_db_silver.db/tb_name_basics/", True)

Out[62]: False

In [0]:
%sql
CREATE OR REPLACE TABLE IMDB_DB_SILVER.tb_name_basics (
  nconst CHAR(10) NOT NULL,
  primaryName VARCHAR(200) NOT NULL,
  birthYear INT NOT NULL,
  deathYear INT NOT NULL,
  primaryProfession VARCHAR(500),
  knownForTitles VARCHAR(500)
);

INSERT INTO IMDB_DB_SILVER.tb_name_basics (
  nconst,
  primaryName,
  birthYear,
  deathYear,
  primaryProfession,
  knownForTitles
)
SELECT 
  nconst, 
  primaryName, 
  CASE WHEN TRIM(birthYear) = '\\N' THEN 0 ELSE birthYear END birthYear,
  CASE WHEN TRIM(deathYear) = '\\N' THEN 0 ELSE deathYear END deathYear,
  primaryProfession,
  knownForTitles
FROM IMDB_DB_BRONZE.tb_name_basics
WHERE primaryProfession IS NOT NULL AND TRIM(primaryProfession) <> '\\N' 
  AND knownForTitles IS NOT NULL AND TRIM(knownForTitles) <> '\\N' ;

-- LISTANDO OS 30 REGISTROS PARA NÃO FICAR MUITO GRANDE NO GITHUB E COMPROVAR A IMPORTAÇÃO
SELECT * FROM IMDB_DB_SILVER.tb_name_basics ORDER BY birthYear DESC LIMIT 30;

nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
nm14249242,Ronnie Lordi,2024,0,"actor,writer,producer",tt23726038
nm6703662,Jerzy Ficowski,2024,2006,"writer,miscellaneous,soundtrack","tt0441074,tt0402621,tt21340576,tt10782220"
nm15136119,Aris A. Stavropoulos,2024,0,producer,tt27242470
nm16698473,Moo Deng,2024,0,archive_footage,tt0044298
nm16864648,Alexandra Ardelyan,2024,0,"writer,editor,director","tt35166273,tt35675660"
nm17086023,Leo Rogic,2023,0,actor,tt31868189
nm15864570,Jimmy Dale Murray,2023,0,actor,tt27567400
nm15263526,Ntombi Hlatswayo,2023,0,set_decorator,tt29144943
nm16458484,Ella Skye Broadfoot,2023,0,actress,tt33247659
nm11786384,Hari Tahov,2022,0,"actor,director,writer","tt12826302,tt21047332,tt32986771,tt27250635"


Validar os dados importados para a camada SILVER

In [0]:
%sql
-- Verificar duplicidade.
-------------------------
SELECT CASE WHEN COUNT(0) > 1 THEN 'Há dados duplicados na tabela' ELSE 'Nenhum dado duplicado foi encontrado com a consulta' END AS Resultado 
FROM (
  SELECT nconst, COUNT(0) TOTAL FROM IMDB_DB_SILVER.tb_name_basics GROUP BY nconst HAVING COUNT(0) > 1
);

Resultado
Nenhum dado duplicado foi encontrado com a consulta


In [0]:
%sql
-- Conferência dasa quantidades de registros
--------------------------------------------
SELECT 
  (SELECT COUNT(0) FROM imdb_db_bronze.tb_name_basics) TOTAL_CAMADA_BRONZE,
  COUNT(0) TOTAL_CAMADA_SILVER, 
  CASE WHEN COUNT(0) <= (SELECT COUNT(0) FROM imdb_db_bronze.tb_name_basics) THEN  'OK' ELSE 'ERRO - QUANTIDADE DE REGISTROS NA CAMADA SILVER MAIOR' END AS VERIFICACAO 
FROM IMDB_DB_SILVER.tb_name_basics;

TOTAL_CAMADA_BRONZE,TOTAL_CAMADA_SILVER,VERIFICACAO
14281080,11066196,OK


###Criando e preenchendo a "tb_title_ratings".

- Colunas da tabela com tipo de acordo com dicionário de dados.
- Somente registros com chave não nula 
- Convertendo as colunas 'numVotes' para o tipo numérico e inteiro e preenchendo com 0 (Zero) se o valor for nulo.

In [0]:
#Removendo vínculo do arquivo da tabela
dbutils.fs.rm("dbfs:/user/hive/warehouse/imdb_db_silver.db/tb_title_ratings/", True)

Out[66]: False

In [0]:
%sql
CREATE OR REPLACE TABLE IMDB_DB_SILVER.tb_title_ratings (
  tconst CHAR(10) NOT NULL,
  averageRating VARCHAR(10) NOT NULL,
  numVotes INT NOT NULL
);

INSERT INTO IMDB_DB_SILVER.tb_title_ratings (
  tconst,
  averageRating,
  numVotes
)
SELECT 
  tconst,
  averageRating,
  numVotes
FROM IMDB_DB_BRONZE.tb_title_ratings
WHERE tconst IS NOT NULL
  AND averageRating IS NOT NULL
  AND numVotes IS NOT NULL;

-- LISTANDO OS 30 REGISTROS PARA NÃO FICAR MUITO GRANDE NO GITHUB E COMPROVAR A IMPORTAÇÃO
SELECT * FROM IMDB_DB_SILVER.tb_title_ratings ORDER BY numVotes LIMIT 30;

tconst,averageRating,numVotes
tt7529184,7.0,5
tt35870745,7.8,5
tt10254894,7.2,5
tt35912619,7.4,5
tt0426808,6.4,5
tt35870747,7.8,5
tt10282784,7.8,5
tt35839851,2.2,5
tt0039738,5.0,5
tt3587436,9.2,5


Validar os dados importados para a camada SILVER

In [0]:
%sql
-- Veriicar duplicidade.
------------------------
SELECT CASE WHEN COUNT(0) > 1 THEN 'Há dados duplicados na tabela' ELSE 'Nenhum dado duplicado foi encontrado com a consulta' END AS Resultado 
FROM (
  SELECT tconst, COUNT(0) TOTAL FROM IMDB_DB_SILVER.tb_title_ratings GROUP BY tconst HAVING COUNT(0) > 1
);

Resultado
Nenhum dado duplicado foi encontrado com a consulta


In [0]:
%sql
-- Verificar se alguma media nao foi preenchida com 3 caracteres [número] [virgula] [número]
SELECT len(averageRating) FROM IMDB_DB_SILVER.tb_title_ratings WHERE len(averageRating) < 3;

len(averageRating)


In [0]:
%sql
-- Conferência dasa quantidades de registros
--------------------------------------------
SELECT 
  (SELECT COUNT(0) FROM imdb_db_bronze.tb_title_ratings) TOTAL_CAMADA_BRONZE,
  COUNT(0) TOTAL_CAMADA_SILVER, 
  CASE WHEN COUNT(0) = (SELECT COUNT(0) FROM imdb_db_bronze.tb_title_ratings) THEN  'OK' ELSE 'ERRO - QUANTIDADE DE REGISTROS NA CAMADA SILVER DIFERENTE' END AS VERIFICACAO 
FROM IMDB_DB_SILVER.tb_title_ratings;

TOTAL_CAMADA_BRONZE,TOTAL_CAMADA_SILVER,VERIFICACAO
1549448,1549448,OK


###Criando e preenchendo a "tb_title_basics".

- Somente registros com chave não nula 
- Convertendo a coluna 'isAdult' para o tipo numérico e inteiro e preenchendo com 0 (Zero) se o valor for nulo.
- Convertendo as colunas 'startYear' e 'endYear' para o tipo numérico e inteiro e preenchendo com 0 (Zero) se o valor for nulo ou com valor "\N" (não aplicável).

In [0]:
#Removendo vínculo do arquivo da tabela
dbutils.fs.rm("dbfs:/user/hive/warehouse/imdb_db_silver.db/tb_title_basics/", True)

Out[71]: False

In [0]:
%sql
CREATE OR REPLACE TABLE IMDB_DB_SILVER.tb_title_basics (
  tconst CHAR(10) NOT NULL,
  titleType VARCHAR(20) NOT NULL,
  primaryTitle VARCHAR(500) NOT NULL,
  originalTitle VARCHAR(500) NOT NULL,
  isAdult INT NOT NULL,
  startYear INT NOT NULL,
  endYear INT NOT NULL,
  runtimeMinutes CHAR(30),
  genres VARCHAR(200)
);

INSERT INTO IMDB_DB_SILVER.tb_title_basics (
  tconst,
  titleType,
  primaryTitle,
  originalTitle,
  isAdult,
  startYear,
  endYear,
  runtimeMinutes,
  genres
)
SELECT 
  tconst,
  titleType,
  primaryTitle,
  originalTitle,
  CASE WHEN TRIM(isAdult) = '\\N' THEN 0 ELSE isAdult END isAdult,
  startYear,
  CASE WHEN TRIM(endYear) = '\\N' THEN 0 ELSE endYear END AS endYear,
  CASE WHEN TRIM(runtimeMinutes) = '\\N' THEN NULL ELSE runtimeMinutes END AS runtimeMinutes,
  genres
FROM IMDB_DB_BRONZE.tb_title_basics
WHERE tconst IS NOT NULL AND TRIM(startYear) <> '\\N' AND TRIM(genres) <> '\\N';

-- LISTANDO OS 30 REGISTROS PARA NÃO FICAR MUITO GRANDE NO GITHUB E COMPROVAR A IMPORTAÇÃO
SELECT * FROM IMDB_DB_SILVER.tb_title_basics ORDER BY startYear, endYear LIMIT 30;

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
tt3155794,short,Passage de Venus,Passage de Venus,0,1874,0,1,"Documentary,History,Short"
tt14495706,short,La Rosace Magique,La Rosace Magique,0,1878,0,1,"Animation,Short"
tt32172647,short,La Glissade,La Glissade,0,1878,0,1,"Animation,Short"
tt27119262,short,Les Chiens Savants,Les Chiens Savants,0,1878,0,1,"Animation,Short"
tt32167266,short,Le Trapèze,Le Trapèze,0,1878,0,1,"Animation,Short"
tt16763674,short,The Tight-rope Dance,La danse sur la corde,0,1878,0,1,"Animation,Short"
tt32169827,short,L'Aquarium,L'Aquarium,0,1878,0,1,"Animation,Short"
tt12592084,short,Le singe musicien,Le singe musicien,0,1878,0,1,"Animation,Short"
tt32169854,short,Le Jongleur,Le Jongleur,0,1878,0,1,"Animation,Short"
tt16763740,short,Skipping Rope,Le jeu de corde,0,1878,0,1,"Animation,Short"


Validar os dados importados para a camada SILVER

In [0]:
%sql
-- Veriicar duplicidade.
------------------------
SELECT CASE WHEN COUNT(0) > 0 THEN 'Há dados duplicados na tabela' ELSE 'Nenhum dado duplicado foi encontrado com a consulta' END AS Resultado 
FROM (
  SELECT tconst, COUNT(0) TOTAL FROM IMDB_DB_SILVER.tb_title_basics GROUP BY tconst HAVING COUNT(0) > 1
);

Resultado
Nenhum dado duplicado foi encontrado com a consulta


In [0]:
%sql
-- Conferência dasa quantidades de registros
--------------------------------------------
SELECT 
  (SELECT COUNT(0) FROM imdb_db_bronze.tb_title_basics) TOTAL_CAMADA_BRONZE,
  COUNT(0) TOTAL_CAMADA_SILVER, 
  CASE WHEN COUNT(0) <= (SELECT COUNT(0) FROM imdb_db_bronze.tb_title_basics) THEN  'OK' ELSE 'ERRO - QUANTIDADE DE REGISTROS NA CAMADA SILVER MAIOR' END AS VERIFICACAO 
FROM IMDB_DB_SILVER.tb_title_basics;

TOTAL_CAMADA_BRONZE,TOTAL_CAMADA_SILVER,VERIFICACAO
11542593,9693050,OK


###Criando e preenchendo a "tb_title_crew".

- Somente registros com chave não nula 
- Tratando os campos que do tipo string (texto) e tenha valor "\N" (não aplicável) com valor [vazio].

In [0]:
#Removendo vínculo do arquivo da tabela
dbutils.fs.rm("dbfs:/user/hive/warehouse/imdb_db_silver.db/tb_title_crew/", True)

Out[75]: False

In [0]:
%sql
-- Obter tamanho maximo no preenchimento dos campos writers e directors para configurar campo na tabela silver
SELECT tamanho_maximo_campo_writers, tamanho_maximo_campo_directors
FROM 
 (SELECT MAX(LEN(writers)) tamanho_maximo_campo_writers FROM IMDB_DB_BRONZE.tb_title_crew),
 (SELECT MAX(LEN(directors)) tamanho_maximo_campo_directors FROM IMDB_DB_BRONZE.tb_title_crew)
 ;

tamanho_maximo_campo_writers,tamanho_maximo_campo_directors
13906,5300


In [0]:
%sql
CREATE OR REPLACE TABLE IMDB_DB_SILVER.tb_title_crew (
  tconst CHAR(10) NOT NULL,
  directors VARCHAR(5300),
  writers VARCHAR(13906)
);

INSERT INTO IMDB_DB_SILVER.tb_title_crew (
  tconst,
  directors,
  writers
)
SELECT
  tconst,
  CASE WHEN TRIM(directors) = '\\N' THEN NULL ELSE directors END AS directors,
  CASE WHEN TRIM(writers) = '\\N' THEN NULL ELSE writers END AS writers
FROM IMDB_DB_BRONZE.tb_title_crew
WHERE TRIM(directors) <> '\\N' AND TRIM(writers) <> '\\N';

-- LISTANDO OS 30 REGISTROS PARA NÃO FICAR MUITO GRANDE NO GITHUB E COMPROVAR A IMPORTAÇÃO
SELECT * FROM IMDB_DB_SILVER.tb_title_crew WHERE directors IS NOT NULL AND writers IS NOT NULL LIMIT 30;

tconst,directors,writers
tt0000003,nm0721526,nm0721526
tt0000009,nm0085156,nm0085156
tt0000015,nm0721526,nm0721526
tt0000036,nm0005690,nm0410331
tt0000076,nm0005690,nm0410331
tt0000091,nm0617588,nm0617588
tt0000108,nm0005690,nm0410331
tt0000109,nm0005690,nm0410331
tt0000110,nm0005690,nm0410331
tt0000111,nm0005690,nm0410331


Validar os dados importados para a camada SILVER

In [0]:
%sql
-- Veriicar duplicidade.
------------------------
SELECT CASE WHEN COUNT(0) > 0 THEN 'Há dados duplicados na tabela' ELSE 'Nenhum dado duplicado foi encontrado com a consulta' END AS Resultado 
FROM (
  SELECT tconst, COUNT(0) TOTAL FROM IMDB_DB_SILVER.tb_title_crew GROUP BY tconst HAVING COUNT(0) > 1
);

Resultado
Nenhum dado duplicado foi encontrado com a consulta


In [0]:
%sql
-- Conferência se há registro com  directors e writers nulos no mesmo registro.
---------------------------------------------
SELECT CASE WHEN COUNT(0) > 0 THEN 'Há dados duplicados na tabela' ELSE 'Nenhum dado duplicado foi encontrado com a consulta' END AS VERIFICACAO 
FROM IMDB_DB_SILVER.tb_title_crew WHERE directors IS NULL AND writers IS NULL;

VERIFICACAO
Nenhum dado duplicado foi encontrado com a consulta


In [0]:
%sql
-- Conferência dasa quantidades de registros
--------------------------------------------
SELECT 
  (SELECT COUNT(0) FROM imdb_db_bronze.tb_title_crew) TOTAL_CAMADA_BRONZE,
  COUNT(0) TOTAL_CAMADA_SILVER, 
  CASE WHEN COUNT(0) <= (SELECT COUNT(0) FROM imdb_db_bronze.tb_title_crew) THEN  'OK' ELSE 'ERRO - QUANTIDADE DE REGISTROS NA CAMADA SILVER MAIOR' END AS VERIFICACAO 
FROM IMDB_DB_SILVER.tb_title_crew;

TOTAL_CAMADA_BRONZE,TOTAL_CAMADA_SILVER,VERIFICACAO
11542593,5014334,OK


###Criando e preenchendo a "tb_title_principals".

- Somente registros com chave não nula 
- Coluna ordering numérica

In [0]:
#Removendo vínculo do arquivo da tabela
dbutils.fs.rm("dbfs:/user/hive/warehouse/imdb_db_silver.db/tb_title_principals/", True)

Out[81]: False

In [0]:
%sql
-- Obter tamanho maximo no preenchimento doS campoS job e caracters para configurar campo na tabela silver
SELECT tamanho_maximo_campo_job, tamanho_maximo_campo_characters
FROM 
  (SELECT MAX(LEN(job)) tamanho_maximo_campo_job FROM IMDB_DB_BRONZE.tb_title_principals),  
  (SELECT MAX(LEN(characters)) tamanho_maximo_campo_characters FROM IMDB_DB_BRONZE.tb_title_principals)
;

tamanho_maximo_campo_job,tamanho_maximo_campo_characters
290,463


In [0]:
%sql
CREATE OR REPLACE TABLE IMDB_DB_SILVER.tb_title_principals (
  tconst CHAR(10) NOT NULL,
  ordering INT NOT NULL,
  nconst CHAR(10),
  category VARCHAR(200),
  job VARCHAR(290),
  characters VARCHAR(463)
);

INSERT INTO IMDB_DB_SILVER.tb_title_principals (
  tconst,
  ordering,
  nconst,
  category,
  job, 
  characters
)
SELECT
  tconst,
  CASE WHEN TRIM(ordering) = '\\N' THEN 0 ELSE ordering END AS ordering,
  nconst,
  category,  
  CASE WHEN TRIM(job) = '\\N' THEN NULL ELSE job END AS job,  
  CASE WHEN TRIM(characters) = '\\N' THEN NULL ELSE characters END AS characters
FROM IMDB_DB_BRONZE.tb_title_principals
WHERE tconst IS NOT NULL;

-- LISTANDO OS 30 REGISTROS PARA NÃO FICAR MUITO GRANDE NO GITHUB E COMPROVAR A IMPORTAÇÃO
SELECT * FROM IMDB_DB_SILVER.tb_title_principals LIMIT 30;

tconst,ordering,nconst,category,job,characters
tt0000001,1,nm1588970,self,null,"[""Self""]"
tt0000001,2,nm0005690,director,null,null
tt0000001,3,nm0005690,producer,producer,null
tt0000001,4,nm0374658,cinematographer,director of photography,null
tt0000002,1,nm0721526,director,null,null
tt0000002,2,nm1335271,composer,null,null
tt0000003,1,nm0721526,director,null,null
tt0000003,2,nm0721526,writer,null,null
tt0000003,3,nm1770680,producer,producer,null
tt0000003,4,nm0721526,producer,producer,null


Validar os dados importados para a camada SILVER

In [0]:
%sql
-- Veriicar duplicidade.
------------------------
SELECT CASE WHEN COUNT(0) > 1 THEN 'Há dados duplicados na tabela' ELSE 'Nenhum dado duplicado foi encontrado com a consulta' END AS Resultado 
FROM (
  SELECT tconst, ordering, nconst, category, job, COUNT(0) TOTAL FROM IMDB_DB_SILVER.tb_title_principals GROUP BY tconst, ordering, nconst, category, job HAVING COUNT(0) > 1
);

Resultado
Nenhum dado duplicado foi encontrado com a consulta


In [0]:
%sql
-- Conferência dasa quantidades de registros
--------------------------------------------
SELECT 
  (SELECT COUNT(0) FROM imdb_db_bronze.tb_title_principals) TOTAL_CAMADA_BRONZE,
  COUNT(0) TOTAL_CAMADA_SILVER, 
  CASE WHEN COUNT(0) = (SELECT COUNT(0) FROM imdb_db_bronze.tb_title_principals) THEN  'OK' ELSE 'ERRO - QUANTIDADE DE REGISTROS NA CAMADA SILVER DIFERENTE' END AS VERIFICACAO 
FROM IMDB_DB_SILVER.tb_title_principals;

TOTAL_CAMADA_BRONZE,TOTAL_CAMADA_SILVER,VERIFICACAO
91642432,91642432,OK


###Criando e preenchendo a "tb_title_episode".

- Somente registros com chave não nula 
- Convertendo as colunas 'numVotes' para o tipo numérico e inteiro e preenchendo com 0 (Zero) se o valor for nulo.

In [0]:
#Removendo vínculo do arquivo da tabela
dbutils.fs.rm("dbfs:/user/hive/warehouse/imdb_db_silver.db/tb_title_episode/", True)

Out[86]: False

In [0]:
%sql
CREATE OR REPLACE TABLE IMDB_DB_SILVER.tb_title_episode (
  tconst CHAR(10) NOT NULL,
  parentTconst CHAR(10),
  seasonNumber INT NOT NULL,
  episodeNumber INT NOT NULL
);

INSERT INTO IMDB_DB_SILVER.tb_title_episode (
  tconst,
  parentTconst,
  seasonNumber,
  episodeNumber
)
SELECT  
  tconst,
  parentTconst,
  REPLACE(TRIM(seasonNumber), '\\N', 0) seasonNumber,
  REPLACE(TRIM(episodeNumber), '\\N', 0) episodeNumber
FROM IMDB_DB_BRONZE.tb_title_episode
WHERE tconst IS NOT NULL;

-- LISTANDO OS 30 REGISTROS PARA NÃO FICAR MUITO GRANDE NO GITHUB E COMPROVAR A IMPORTAÇÃO
SELECT * FROM IMDB_DB_SILVER.tb_title_episode LIMIT 30;

tconst,parentTconst,seasonNumber,episodeNumber
tt0031458,tt32857063,0,0
tt0041951,tt0041038,1,9
tt0042816,tt0989125,1,17
tt0042889,tt0989125,0,0
tt0043426,tt0040051,3,42
tt0043631,tt0989125,2,16
tt0043693,tt0989125,2,8
tt0043710,tt0989125,3,3
tt0044093,tt0959862,1,6
tt0044668,tt0044243,2,16


Validar os dados importados para a camada SILVER

In [0]:
%sql
-- Veriicar duplicidade.
------------------------
SELECT CASE WHEN COUNT(0) > 1 THEN 'Há dados duplicados na tabela' ELSE 'Nenhum dado duplicado foi encontrado com a consulta' END AS Resultado 
FROM (
  SELECT tconst, COUNT(0) TOTAL FROM IMDB_DB_SILVER.tb_title_episode GROUP BY tconst HAVING COUNT(0) > 1
);

Resultado
Nenhum dado duplicado foi encontrado com a consulta


In [0]:
%sql
-- Conferência dasa quantidades de registros
--------------------------------------------
SELECT 
  (SELECT COUNT(0) FROM imdb_db_bronze.tb_title_episode) TOTAL_CAMADA_BRONZE,
  COUNT(0) TOTAL_CAMADA_SILVER, 
  CASE WHEN COUNT(0) = (SELECT COUNT(0) FROM imdb_db_bronze.tb_title_episode) THEN  'OK' ELSE 'ERRO - QUANTIDADE DE REGISTROS NA CAMADA SILVER DIFERENTE' END AS VERIFICACAO 
FROM IMDB_DB_SILVER.tb_title_episode;

TOTAL_CAMADA_BRONZE,TOTAL_CAMADA_SILVER,VERIFICACAO
8880703,8880703,OK
